In [ ]:
import math
from dataclasses import dataclass
from typing import Callable, Dict, List, Optional, Tuple, Union

import sympy as sp
from sympy import symbols, log, integrate, simplify

# =========================================
#  Teaching Step System (preserves original logic)
# =========================================
class PedagogicalStep:
    def __init__(self, level: int, title: str, expression: str,
                 explanation: str = "", step_type: str = "calculation", 
                 intuition: str = "", visual_aid: str = ""):
        self.level = level
        self.title = title
        self.expression = expression
        self.explanation = explanation
        self.step_type = step_type
        self.intuition = intuition
        self.visual_aid = visual_aid

    def __str__(self):
        indent = "  " * self.level
        tag = {"insight": "💡", "pattern": "🔍", "conclusion": "✅", 
               "visualization": "🎨", "method": "🛠️"}.get(self.step_type, "🔢")
        
        out = f"\n{indent}{tag} {self.title}\n{indent}  {self.expression}"
        
        if self.intuition:
            out += f"\n{indent}  🧠 Intuition: {self.intuition}"
        if self.visual_aid:
            out += f"\n{indent}  📊 Visual: {self.visual_aid}"
        if self.explanation:
            out += f"\n{indent}  💭 {self.explanation}"
        return out

# =========================================
#  visualization (preserves original tree logic)
# =========================================
class VisualizationHelper:
    @staticmethod
    def create_recursion_tree(levels: int, branches: List, name: str = "T"):
        """ version of original tree with intuitive annotations"""
        n = symbols('n', positive=True)
        txt = [f"🌳 RECURSION TREE - How work spreads across levels:"]
        txt.append(f"Level 0: {name}(n) ← original problem")
        
        curr = [(name, n)]
        for lv in range(1, levels + 1):
            nxt = []
            row = []
            for _, arg in curr:
                for b in branches:
                    node = f"{name}({sp.simplify(b.phi.subs({n: arg}))})"
                    row.append(node)
                    nxt.append((name, sp.simplify(b.phi.subs({n: arg}))))
            if len(row) <= 5:
                txt.append("  " * lv + "  ".join(row))
            else:
                txt.append("  " * lv + f"{row[0]}  {row[1]}  ...  {row[-1]}  ({len(row)} subproblems)")
            curr = nxt
        return "\n".join(txt)

    @staticmethod 
    def method_intuition(method_name: str) -> str:
        """Intuitive explanations for each method"""
        explanations = {
            "Master": "Compare recursive tree growth vs per-level work",
            "Akra-Bazzi": "Generalized balance across multiple branches", 
            "Subtractive": "Sum work as we peel off layers: T(n) + T(n-1) + ...",
            "Power-shrink": "Very fast shrinking: n → n^r → n^(r²) → ... in log log n steps",
            "Iterated": "Slow logarithmic descent: n → log n → log log n → ...",
            "Dominance": "One term completely overwhelms the recursive structure"
        }
        return explanations.get(method_name, "Analyzing recurrence structure")

# =========================================
# Preserve original recurrence structure exactly
# =========================================
@dataclass
class Branch:
    a: Union[int, float, sp.Expr]   # multiplier of T(phi(n))
    phi: sp.Expr                    # subproblem size function

class GeneralRecurrence:
    """Exactly the same as original - no changes to core logic"""
    def __init__(self,
                 branches: List[Branch],
                 g: Union[int, float, sp.Expr],
                 base_cases: Optional[Dict[int, Union[int, float]]] = None,
                 name: str = "T"):
        self.n = symbols('n', positive=True)
        self.branches = branches
        self.g = g if isinstance(g, sp.Expr) else sp.Integer(g)
        self.base_cases = base_cases or {1: 1}
        self.name = name
        self._validate()

    def _validate(self):
        if not self.branches and self.g is None:
            raise ValueError("At least one branch or a non-recursive term g(n) is required.")
        for b in self.branches:
            if sp.simplify(b.phi - self.n) == 0:
                raise ValueError("Branch phi(n) == n would not shrink the problem.")

    def __str__(self):
        n = self.n
        parts = []
        for b in self.branches:
            a_str = str(b.a)
            phi_str = str(b.phi)
            parts.append(f"{a_str}·{self.name}({phi_str})")
        rhs = " + ".join(parts) if parts else ""
        if self.g is not None:
            rhs = (rhs + " + " if rhs else "") + str(self.g)
        base_str = ", ".join([f"{self.name}({k}) = {v}" for k, v in self.base_cases.items()])
        return f"{self.name}(n) = {rhs};  {base_str}"

# =========================================
#  solver - preserves ALL original mathematical logic
# =========================================
class PedagogicalSolver:
    """Keeps original solver logic exactly, adds pedagogical layer"""
    def __init__(self, show_patterns=True, show_insights=True, max_tree_levels=3):
        self.n = symbols('n', positive=True)
        self.steps: List[PedagogicalStep] = []
        self.show_patterns = show_patterns
        self.show_insights = show_insights
        self.max_tree_levels = max_tree_levels

    def _add(self, lvl, title, expr, expl="", step_type="calculation", intuition="", visual=""):
        self.steps.append(PedagogicalStep(lvl, title, expr, expl, step_type, intuition, visual))

    def _theta(self, s: str) -> str:
        return f"Θ({s})"

    def _classify_integral_growth(self, g_expr: sp.Expr, p_val: float) -> str:
        """EXACT copy of original integral growth classifier"""
        u = sp.symbols('u', positive=True)
        g = simplify(g_expr.subs({self.n: u}))

        # Attempt to match g(u) = C * u^k * log(u)^m
        k = None
        m = 0
        C = sp.Integer(1)

        try:
            k = sp.degree(sp.together(g), gen=u)
        except Exception:
            k = None

        # Count log factors
        log_power = 0
        tmp = g
        if tmp.has(sp.log(u)):
            tmp = sp.simplify(tmp)
            if tmp.func == sp.Pow and tmp.args[0] == sp.log(u) and tmp.args[1].is_Number:
                log_power = int(tmp.args[1])
            else:
                try:
                    for cand_m in range(0, 6):
                        if sp.simplify(g / (sp.log(u) ** cand_m)).has(sp.log(u)):
                            continue
                        log_power = cand_m
                        break
                except Exception:
                    pass
            m = log_power

        if k is None:
            try:
                k = sp.LT(sp.series(g, u, sp.oo, 1)).as_leading_term(u).as_powers_dict().get(u, 0)
                k = float(k)
            except Exception:
                k = None

        if k is None:
            return "integral term (unsimplified)"

        k = float(k)
        p = float(p_val)
        alpha = k - (p + 1)
        
        if alpha < -1 - 1e-9:
            return "constant"
        elif abs(alpha + 1) <= 1e-9:
            if m == 0:
                return "log n"
            else:
                return f"(log n)^{m+1}"
        else:
            power = alpha + 1
            if abs(power - 1) <= 1e-9 and m == 0:
                return "n"
            if m == 0:
                return f"n^{power:.3f}"
            else:
                return f"n^{power:.3f} (log n)^{m}"

    def _try_master(self, R: GeneralRecurrence) -> Optional[Tuple[str, str]]:
        """EXACT copy of original Master Theorem logic with  explanations"""
        if len(R.branches) != 1:
            return None

        bch = R.branches[0]
        n = self.n

        try:
            a_val = float(bch.a) if isinstance(bch.a, (int, float)) or (isinstance(bch.a, sp.Expr) and bch.a.is_number) else None
            phi = sp.simplify(bch.phi)
            if not phi.has(n):
                return None
            c_sym = sp.simplify(sp.diff(phi, n))
            if sp.simplify(phi - c_sym * n) != 0:
                return None
            c_val = float(c_sym) if (isinstance(c_sym, (sp.Integer, sp.Float)) or c_sym.is_Number) else None
        except Exception:
            return None

        if a_val is None or c_val is None or not (0.0 < c_val < 1.0) or a_val < 1.0:
            return None

        b_val = 1.0 / c_val
        p = math.log(a_val) / math.log(b_val)
        f = sp.simplify(R.g)

        #  pedagogical intro
        self._add(1, "Master Theorem (single branch)",
                 f"Form: T(n) = a·T(n/b) + f(n) with a={a_val:g}, b={b_val:g}.",
                 "Threshold function: n^{log_b a}. We compare f(n) to n^{log_b a}.",
                 "insight",
                 intuition="Single recursive branch - compare tree growth vs work per level")

        # Show tree structure visually
        tree_analysis = f"""
Level 0: 1 call of size n, work = f(n)
Level 1: {int(a_val)} calls of size n/{int(b_val)}, total work = {int(a_val)}·f(n/{int(b_val)})
Level 2: {int(a_val**2)} calls of size n/{int(b_val**2)}, total work = {int(a_val**2)}·f(n/{int(b_val**2)})
...
Depth: log_{int(b_val)}(n), Leaves: {int(a_val)}^(log_{int(b_val)} n) = n^{p:.3f}
        """
        
        self._add(2, "Tree Analysis", tree_analysis.strip(),
                 step_type="visualization",
                 intuition="Each level multiplies subproblems, but shrinks their size")

        self._add(2, "Threshold", 
                 f"n^{{log_b a}} = n^{p} with p = log_{b_val:g}({a_val:g}) = {p:.6f}",
                 "This is the critical polynomial growth to compare against f(n).",
                 "calculation",
                 intuition="This p is the natural growth rate from recursive structure alone")

        # EXACT same case analysis as original
        ratio = sp.simplify(f / (n**p))
        k_detected = None
        try:
            for k in range(0, 6):
                L = sp.limit(ratio / (sp.log(n)**k), n, sp.oo)
                if L.is_finite and L != 0:
                    k_detected = k
                    break
        except Exception:
            pass

        case = None
        if k_detected is not None:
            case = 2
        else:
            try:
                L0 = sp.limit(ratio, n, sp.oo)
                if L0 == 0:
                    case = 1
                elif L0 == sp.oo:
                    case = 3
            except Exception:
                return None

        # Same conclusions as original, with better explanations
        if case == 1:
            ans = self._theta(f"n^{p:.6f}")
            self._add(3, "Case 1 (f smaller)", f"f(n) = O(n^{p-ε})",
                     "Master Thm Case 1 ⇒ T(n)=Θ(n^{log_b a}).", "insight",
                     intuition="Work per level is too small - recursive tree dominates")
            self._add(5, "Final Complexity", ans, "By Master Theorem (Case 1).", "conclusion")
            return ans, "Master"

        if case == 2:
            ans = self._theta(f"n^{p:.6f} (log n)^{k_detected+1}")
            self._add(3, "Case 2 (f matches)", f"f(n) = Θ(n^{p} (log n)^{k}) with k={k_detected}",
                     "Master Thm Case 2 ⇒ multiply by an extra log.", "insight",
                     intuition="Perfect balance between work and tree growth - log factor appears")
            self._add(5, "Final Complexity", ans, "By Master Theorem (Case 2).", "conclusion")
            return ans, "Master"

        if case == 3:
            ans = self._theta(str(f))
            self._add(3, "Case 3 (f larger)",
                     "f(n) = Ω(n^{p+ε}) and a·f(n/b) ≤ c·f(n) (regularity)",
                     "Under the regularity condition, Master Thm Case 3 ⇒ T(n)=Θ(f(n)).",
                     "insight",
                     intuition="Work per level grows faster than tree - work dominates")
            self._add(5, "Final Complexity", ans, "By Master Theorem (Case 3), assuming regularity.", "conclusion")
            return ans, "Master"

        return None

    def _try_akra_bazzi(self, R: GeneralRecurrence) -> Optional[Tuple[str, str]]:
        """EXACT copy of original Akra-Bazzi with  pedagogy"""
        n = self.n
        bs, as_ = [], []
        for b in R.branches:
            phi = sp.simplify(b.phi)
            if not phi.has(n):
                return None
            coeff = sp.simplify(sp.diff(phi, n))
            if sp.simplify(phi - coeff * n) != 0:
                return None
            if not (isinstance(coeff, (sp.Integer, sp.Float)) or coeff.is_Number):
                return None
            c = float(coeff)
            if not (0.0 < c < 1.0):
                return None
            if not (isinstance(b.a, (int, float)) or (isinstance(b.a, sp.Expr) and b.a.is_number)):
                return None
            a_val = float(b.a)
            if a_val <= 0:
                return None
            bs.append(c)
            as_.append(a_val)

        if not bs:
            return None

        self._add(1, "Akra–Bazzi Method",
                 "Form: T(n) = Σ a_i T(b_i n) + g(n),  with constants a_i>0 and 0<b_i<1.",
                 "Conclusion: T(n) = Θ(n^p [1 + ∫_1^n g(u)/u^{p+1} du]), where p solves Σ a_i b_i^p = 1.",
                 "insight",
                 intuition="Generalizes Master Theorem for multiple branches or irregular coefficients")

        # Show the balance equation clearly
        balance_terms = [f"{as_[i]:.3g}×({bs[i]:.3g})^p" for i in range(len(as_))]
        balance_eq = " + ".join(balance_terms) + " = 1"
        
        self._add(2, "Balance Equation", f"Solve: {balance_eq}",
                 step_type="calculation",
                 intuition="Find p where total recursive 'weight' equals 1 across all levels")

        # EXACT same p solving as original
        p = sp.symbols('p', real=True)
        eq = sp.Eq(sum(as_[i] * (bs[i] ** p) for i in range(len(bs))), 1.0)
        p_sol = None
        for guess in [0.0, 0.5, 1.0, 2.0]:
            try:
                p_sol = float(sp.nsolve(eq, guess))
                break
            except Exception:
                continue
        if p_sol is None:
            return None

        self._add(2, "Exponent Equation",
                 f"Σ a_i b_i^p = 1  ⇒  p ≈ {p_sol:.6f}",
                 "This p balances total work across levels.",
                 "calculation")

        # EXACT same integral classification
        u = sp.symbols('u', positive=True)
        g_u = sp.simplify(sp.sympify(R.g).subs({R.n: u}))
        growth_txt = "integral term (unsimplified)"
        try:
            growth_txt = self._classify_integral_growth(R.g, p_sol)
        except Exception:
            pass

        self._add(2, "Akra–Bazzi Integral (growth)",
                 f"∫_1^n g(u)/u^{p+1} du  ~  {growth_txt}",
                 "This modifies the n^p factor when g is large (equal or dominant case).",
                 "calculation",
                 intuition="Integral captures how non-recursive work accumulates across all levels")

        # EXACT same final result logic as original
        g = sp.simplify(R.g)
        if growth_txt == "constant":
            ans = self._theta(f"n^{p_sol:.6f}")
        elif growth_txt.startswith("n^"):
            ans = self._theta(f"n^{p_sol:.6f} · (1 + {growth_txt})")
        else:
            ans = self._theta(f"n^{p_sol:.6f} · (1 + {growth_txt})")

        # Special cases from original
        if g == 1:
            ans = self._theta(f"n^{p_sol:.6f}")
        if g == R.n and abs(p_sol - 1.0) < 1e-8:
            ans = self._theta("n log n")
        if g == R.n**2 and abs(p_sol - 2.0) < 1e-8:
            ans = self._theta("n^2 log n")

        self._add(5, "Final Complexity", ans,
                 "By the Akra–Bazzi method (a generalization of the Master Theorem).",
                 "conclusion")
        return ans, "Akra–Bazzi"

    def _try_subtractive(self, R: GeneralRecurrence) -> Optional[Tuple[str, str]]:
        """EXACT copy of original subtractive logic"""
        if len(R.branches) != 1:
            return None
        b = R.branches[0]
        if not (isinstance(b.a, (int, float)) or (isinstance(b.a, sp.Expr) and b.a.is_number)):
            return None
        if float(b.a) != 1.0:
            return None

        phi = sp.simplify(b.phi)
        n = self.n
        c = sp.simplify(n - phi)
        if not (isinstance(c, (sp.Integer, sp.Float)) or c.is_Number):
            return None
        c = float(c)
        if c <= 0:
            return None

        self._add(1, "Subtractive Recurrence",
                 f"Form: {R.name}(n) = {R.name}(n-{int(c)}) + g(n)",
                 "Unroll: T(n) = Σ_{i} g(n - i c) + base  ≈ (1/c) ∫_0^n g(u) du.",
                 "insight",
                 intuition="Like peeling layers off an onion - constant size reduction each step")

        # Show unrolling pattern
        unroll_visual = f"""
T(n) = T(n-{int(c)}) + g(n)
     = T(n-{int(2*c)}) + g(n-{int(c)}) + g(n)
     = T(n-{int(3*c)}) + g(n-{int(2*c)}) + g(n-{int(c)}) + g(n)
     = ...
     = base + g({int(c)}) + g({int(2*c)}) + ... + g(n)
     ≈ ∫₁ⁿ g(u) du
        """
        
        self._add(2, "Unrolling Pattern", unroll_visual.strip(),
                 step_type="visualization", 
                 intuition="Sum telescopes to integral of work function")

        g = sp.simplify(R.g)
        u = sp.symbols('u', positive=True)
        ans_txt = "∫ g(u) du (growth)"
        try:
            F = integrate(g.subs({R.n: u}), (u, 1, R.n))
            F_lead = sp.simplify(F).leading_term(R.n)
            ans_txt = self._theta(str(F_lead))
        except Exception:
            # EXACT same fallbacks as original
            if g == 1:
                ans_txt = self._theta("n")
            elif g == R.n:
                ans_txt = self._theta("n^2")
            elif g == R.n**2:
                ans_txt = self._theta("n^3")
            elif g == R.n * sp.log(R.n):
                ans_txt = self._theta("n^2 log n")
            else:
                ans_txt = self._theta("∫ g")

        self._add(5, "Final Complexity", ans_txt,
                 "By unrolling as a finite sum (integral approximation).",
                 "conclusion")
        return ans_txt, "Subtractive"

    def _try_power_shrink(self, R: GeneralRecurrence) -> Optional[Tuple[str, str]]:
        """EXACT copy of original power shrinking logic"""
        if len(R.branches) != 1:
            return None
        b = R.branches[0]
        phi = sp.simplify(b.phi)
        n = self.n

        # detect phi = n**r
        if phi.func == sp.Pow and phi.args[0] == n and phi.args[1].is_Number:
            r = float(phi.args[1])
        else:
            return None
        if not (0.0 < r < 1.0):
            return None

        a_n = sp.simplify(b.a)
        g = sp.simplify(R.g)

        self._add(1, "Power-Shrink Recurrence",
                 f"Form: {R.name}(n) = a(n) T(n^{r}) + g(n) with 0<r<1",
                 "Change variable n = b^{a^k}. Depth in new variable is Θ(log log n).",
                 "insight",
                 intuition="Super-fast shrinking: n → n^r → n^(r²) → n^(r³) → ... hits 1 very quickly")

        # EXACT r = 1/2 analysis from original
        if abs(r - 0.5) < 1e-12 and (isinstance(a_n, (int, float)) or a_n == sp.sqrt(n) or a_n == n**sp.Rational(1,2)):
            self._add(2, "Change of Variables",
                     "Set n = 2^{2^k}, then √n = 2^{2^{k-1}}.",
                     "Define S(k) := T(2^{2^k}). Depth in k becomes linear; original depth ≈ log log n.",
                     "calculation",
                     intuition="Double exponential substitution makes the recursion depth manageable")
            
            if a_n == 1 or (isinstance(a_n, (int, float)) and float(a_n) > 0):
                ans = self._theta("n")
                self._add(5, "Final Complexity", ans,
                         "Additive n dominates across ~log log n levels.", "conclusion")
                return ans, "Power-shrink (r=1/2)"
            if a_n == sp.sqrt(n) or a_n == n**sp.Rational(1,2):
                ans = self._theta("n · log log n")
                self._add(5, "Final Complexity", ans,
                         "√n multiplier accumulates over ~log log n levels: √n × √(√n) × ... creates log log n factor.",
                         "conclusion")
                return ans, "Power-shrink (r=1/2)"

        # EXACT same generic heuristic as original
        if (isinstance(a_n, (int, float)) or (isinstance(a_n, sp.Expr) and a_n.is_Number)):
            a_val = float(a_n)
            if g == n:
                ans = self._theta("n")
            else:
                ans = self._theta(f"{str(g)} · log log n")
            self._add(5, "Final Complexity", ans,
                     "Heuristic for constant a and 0<r<1: depth ≈ log log n.", "conclusion")
            return ans, "Power-shrink"

        return None

    def _try_iterated(self, R: GeneralRecurrence) -> Optional[Tuple[str, str]]:
        """EXACT copy of original iterated argument logic"""
        if len(R.branches) != 1:
            return None
        b = R.branches[0]
        phi = sp.simplify(b.phi)
        a_n = sp.simplify(b.a)
        g = sp.simplify(R.g)
        n = self.n

        if (a_n == 1 or (isinstance(a_n, (int, float)) and float(a_n) == 1.0)) and (g.is_Number and float(g) >= 0.0):
            if phi == sp.log(n) or sp.simplify(phi - sp.log(n)) == 0:
                self._add(1, "Iterated-Argument Recurrence",
                         "Form: T(n) = T(log n) + O(1)",
                         "Each step applies a logarithm; the number of steps is log* n (iterated log).",
                         "insight",
                         intuition="Very slow shrinking: n → log n → log log n → ... takes log* n steps")
                
                depth_visual = """
Step 0: n
Step 1: log n
Step 2: log log n  
Step 3: log log log n
...
Step k: log^(k) n ≤ 2 (base case)
Number of steps: log* n (iterated logarithm)
                """
                
                self._add(2, "Descent Pattern", depth_visual.strip(),
                         step_type="visualization",
                         intuition="Each logarithm shrinks very slowly - takes many steps to reach constant")
                
                ans = self._theta("log* n")
                self._add(5, "Final Complexity", ans,
                         "By counting the number of logarithm applications until reaching the base.", "conclusion")
                return ans, "Iterated"
            if phi == sp.log(sp.log(n)) or sp.simplify(phi - sp.log(sp.log(n))) == 0:
                self._add(1, "Iterated-Argument Recurrence",
                         "Form: T(n) = T(log log n) + O(1)",
                         "Two logs per step, still Θ(log* n) up to a constant factor.",
                         "insight",
                         intuition="Double logarithm per step, but still very slow overall")
                ans = self._theta("log* n")
                self._add(5, "Final Complexity", ans,
                         "By iterated logarithm depth analysis.", "conclusion")
                return ans, "Iterated"
        return None

    def _try_superexp_g_dominance(self, R: GeneralRecurrence) -> Optional[Tuple[str, str]]:
        """EXACT copy of original dominance analysis"""
        if len(R.branches) != 1:
            return None
        bch = R.branches[0]
        n = self.n

        # phi(n) = n/b (constant b>1)
        phi = sp.simplify(bch.phi)
        if not phi.has(n):
            return None
        coeff = sp.simplify(sp.diff(phi, n))
        if sp.simplify(phi - coeff * n) != 0:
            return None
        try:
            c_val = float(coeff)
        except Exception:
            return None
        if not (0.0 < c_val < 1.0):
            return None
        b_val = 1.0 / c_val

        # a(n) = C^{n} (C>1)
        a_n = sp.simplify(bch.a)
        C = None
        if a_n.func == sp.Pow and a_n.args[0].is_Number and a_n.args[1] == n:
            C = float(a_n.args[0])
        elif a_n == 2**n or a_n == sp.Integer(2)**n:
            C = 2.0
        if C is None or C <= 1.0:
            return None

        # g(n) = n^{α n} * (poly/ polylog factors)
        g = sp.simplify(R.g)
        try:
            base, expo = sp.powdenest(g, force=True).as_base_exp()
        except Exception:
            base, expo = g.as_base_exp() if hasattr(g, "as_base_exp") else (None, None)
        alpha = None
        if base == n and expo is not None and expo.has(n):
            coeff_alpha = sp.simplify(sp.diff(expo, n))
            if coeff_alpha.is_Number and float(coeff_alpha) > 0:
                alpha = float(coeff_alpha)
        if alpha is None or alpha <= 0:
            return None  # don't claim dominance if g isn't n^{α n}-type

        # EXACT same pedagogy and conclusion as original
        self._add(1, "Dominance via Unrolling",
                 f"phi(n)=n/{b_val:g}, a(n)={C:g}^n, g(n)=n^{alpha}^n with α≈{alpha:.3g}",
                 "Unrolling to the base gives a product multiplier Π a(n/ b^j) "
                 "≤ C^{n(1 + 1/2 + 1/4 + …)} < C^{2n} = (C^2)^n.",
                 "insight",
                 intuition="Superexponential work function completely overwhelms exponential recursive terms")
        
        dominance_visual = f"""
Recursive contribution: ∏ a(n/b^j) ≤ C^(n + n/2 + n/4 + ...) = C^(2n)
Work function: g(n) = n^n grows much faster than any exponential
Since n^n >> C^(2n) for any constant C, work dominates completely
        """
        
        self._add(2, "Growth Comparison", 
                 "Base term ≤ C^{2n} · T(1) = O((C^2)^n), while g(n)=n^n.",
                 "Since n^n ≫ (C^2)^n (because log n grows unbounded), g(n) dominates.",
                 "calculation",
                 visual=dominance_visual.strip())
        self._add(2, "Other Summands",
                 "Each unrolled g(n/b^j) term for j≥1 is ≤ exp(-Θ(n log n)) times n^n.",
                 "Hence the sum of all j≥1 terms is o(n^n); the j=0 term equals n^n.",
                 "calculation")
        ans = self._theta("n^n")
        self._add(5, "Final Complexity", ans,
                 "By dominance: T(n) = n^n · (1 + o(1)).", "conclusion")
        return ans, "Dominance"

    def solve(self, R: GeneralRecurrence, show_tree=True) -> Tuple[List[PedagogicalStep], str]:
        """EXACT same solving logic as original with  presentation"""
        self.steps = []
        self._add(0, "Problem", str(R),
                 "Trying: Master Theorem → Akra–Bazzi → Subtractive → Power-shrink → Iterated → Superexp Dominance → Fallback.",
                 "insight")
        
        if show_tree and R.branches:
            tree = VisualizationHelper.create_recursion_tree(self.max_tree_levels, R.branches, R.name)
            self._add(0, "Recursion Tree (first levels)", tree, step_type="visualization",
                     intuition="See how subproblems multiply and shrink across levels")

        # EXACT same method order and logic as original
        for method_func in [self._try_master, self._try_akra_bazzi, self._try_subtractive, 
                           self._try_power_shrink, self._try_iterated, self._try_superexp_g_dominance]:
            res = method_func(R)
            if res is not None:
                ans, method_name = res
                # Add method summary with intuition
                self._add(0, f"✅ Solution Method: {method_name}",
                         VisualizationHelper.method_intuition(method_name),
                         step_type="method")
                return self.steps, ans

        self._add(5, "Fallback",
                 "Could not match a closed-form strategy.",
                 "Try giving a hint (e.g., 'Akra–Bazzi', 'n=2^{2^k}', or 'n=b^{a^k}').",
                 "insight")
        return self.steps, "Unresolved"

# =========================================
# EXACT same convenience builders as original
# =========================================
def AB(a_list: List[float], b_list: List[float], g, base={1:1}, name="T") -> GeneralRecurrence:
    n = symbols('n', positive=True)
    branches = [Branch(a=a_list[i], phi=b_list[i]*n) for i in range(len(a_list))]
    return GeneralRecurrence(branches, g, base, name)

def SUB(c: int, g, base={1:1}, name="T") -> GeneralRecurrence:
    n = symbols('n', positive=True)
    return GeneralRecurrence([Branch(1, n - c)], g, base, name)

def POW(a, r: float, g, base={1:1}, name="T") -> GeneralRecurrence:
    n = symbols('n', positive=True)
    a_expr = a if isinstance(a, sp.Expr) else sp.Integer(a) if isinstance(a, int) else sp.Float(a)
    return GeneralRecurrence([Branch(a_expr, n**sp.Float(r))], g, base, name)

def ITER(phi_expr, g, base={1:1}, name="T") -> GeneralRecurrence:
    n = symbols('n', positive=True)
    return GeneralRecurrence([Branch(1, sp.sympify(phi_expr))], g, base, name)

# =========================================
# Test on EXACT same 12 problems as original
# =========================================
if __name__ == "__main__":
    n = symbols('n', positive=True)
    solver = PedagogicalSolver()

    # EXACT same 12 problems as your original
    problems: List[Tuple[str, GeneralRecurrence]] = [
        ("1) T(n) = T(n-2) + n²", SUB(2, n**2)),
        ("2) T(n) = 3T(n/4) + n log n", AB([3.0], [1/4], n*sp.log(n))),
        ("3) T(n) = T(n/3)+T(n/6)+T(n/2) + n", AB([1.0, 1.0, 1.0], [1/3, 1/6, 1/2], n)),
        ("4) T(n) = √n·T(√n) + n", POW(sp.sqrt(n), 0.5, n)),
        ("5) T(n) = 2ⁿ T(n/2) + nⁿ", GeneralRecurrence([Branch(2**n, n/2)], n**n)),
        ("6) T(n) = 4T(n/2) + n² log n", AB([4.0], [1/2], n**2*sp.log(n))),
        ("7) T(n) = T(n-1) + n²", SUB(1, n**2)),
        ("8) T(n) = 9T(n/3) + n²", AB([9.0], [1/3], n**2)),
        ("9) T(n) = T(n^(1/3)) + 1", POW(1, 1/3, 1)),
        ("10) T(n) = 2T(√n) + log n", POW(2, 0.5, sp.log(n))),
        ("11) T(n) = T(n/3) + T(2n/3) + √n", AB([1.0, 1.0], [1/3, 2/3], sp.sqrt(n))),
        ("12) T(n) = T(log log n) + 1", ITER(sp.log(sp.log(n)), 1)),
    ]

    for i, (title, recurrence) in enumerate(problems, 1):
        print(f"\n{'='*72}")
        print(f"PROBLEM {i}: {title}")
        print('='*72)
        
        steps, answer = solver.solve(recurrence, show_tree=True)
        for step in steps:
            print(step)
        
        print(f"\nFINAL RESULT: {answer}")
        
        if i < len(problems):
            print("\n" + "-" * 72)

PEDAGOGICAL RECURRENCE SOLVER

PROBLEM 1: 1) T(n) = T(n-2) + n²

💡 Problem
  T(n) = 1·T(n - 2) + n**2;  T(1) = 1
  💭 Trying: Master Theorem → Akra–Bazzi → Subtractive → Power-shrink → Iterated → Superexp Dominance → Fallback.

🎨 Recursion Tree (first levels)
  🌳 RECURSION TREE - How work spreads across levels:
Level 0: T(n) ← original problem
  T(n - 2)
    T(n - 4)
      T(n - 6)
  🧠 Intuition: See how subproblems multiply and shrink across levels

  💡 Subtractive Recurrence
    Form: T(n) = T(n-2) + g(n)
    🧠 Intuition: Like peeling layers off an onion - constant size reduction each step
    💭 Unroll: T(n) = Σ_{i} g(n - i c) + base  ≈ (1/c) ∫_0^n g(u) du.

    🎨 Unrolling Pattern
      T(n) = T(n-2) + g(n)
     = T(n-4) + g(n-2) + g(n)
     = T(n-6) + g(n-4) + g(n-2) + g(n)
     = ...
     = base + g(2) + g(4) + ... + g(n)
     ≈ ∫₁ⁿ g(u) du
      🧠 Intuition: Sum telescopes to integral of work function

          ✅ Final Complexity
            Θ(n^3)
            💭 By unrolling as a